In [13]:
import numpy as np
from scipy.integrate import quad
from scipy.optimize import bisect

In [14]:
# ============================
# Constants
# ============================

kpcincm = 3.0857e21       # cm / kpc
MSuninGeV = 1.115e57      # GeV
rSun = 8.277              # kpc
rhoSun = 0.4              # GeV / cm^3
r200 = 220.2              # kpc

In [15]:
# ============================
# DM profile
# ============================

def rhogNFW(r, rhos, rs, gamma):
    x = r / rs
    return rhos / (x**gamma * (1 + x)**(3 - gamma))

In [ ]:
# ============================
# Local DM density constraint
# ============================

def rhos(rho_func, rs, *args):
    return rhoSun / rho_func(rSun, 1.0, rs, *args)

# ============================
# Halo mass constraint
# ============================

def DeltaM200(rho_func, rs, *args):
    rhos_val = rhos(rho_func, rs, *args)

    def integrand(r):
        return r**2 * rho_func(r, rhos_val, rs, *args)
    
    M200 = 4 * np.pi * quad(integrand, 0, r200, epsrel=1e-5)[0]
    M200 *= kpcincm**3 / MSuninGeV
    return M200 - 1e12

def rs(rho_func, *args):
    return bisect(lambda rs, *args: DeltaM200(rho_func, rs, *args), 0.1, 50, args=(args), xtol=1e-6)

13.630477593839169

In [23]:
# ============================
# Line-of-sight integral
# ============================

def J_los(theta, rho_func, *args, D=rSun):
    costh = np.cos(theta)

    rs_   = rs(rho_func, *args)
    rhos_ = rhos(rho_func, rs_, *args)

    def integrand(s):
        r = np.sqrt(D**2 + s**2 - 2*s*D*costh)
        return rho_func(r, rhos_, rs_, *args)**2

    return kpcincm * quad(integrand, 0, 100.0, epsrel=1e-4)[0]

# ============================
# J-factor
# ============================

def Jfactor(thetamax, rho_func, *args, D=rSun):

    Nth  = 200
    ths  = np.radians(np.linspace(0, thetamax, Nth))
    dth  = np.radians((thetamax - 0) / Nth)

    J = 0.0
    for th in ths:
        J += 2*np.pi * np.sin(th) * J_los(th, rho_func, *args, D=D) * dth

    return J

# ============================
# Compute J-factor
# ============================

gamma = 1.26
thetamax = 20
print("J-factor (GeV2/cm5):", Jfactor(thetamax, rhogNFW, gamma))

/var/folders/m5/ptmv1tqj29v601gqzb56cpx80000gn/T/ipykernel_8515/204342538.py:15: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  return kpcincm * quad(integrand, 0, 100.0, epsrel=1e-4)[0]


J-factor (GeV2/cm5): 2.1243967789089963e+23
